In [ ]:
# Блок 1. Загрузка набора данных и извлечение изображений из архива
# Для обучения сети Вы можете использовать собственные наборы данных.
# Подготовьте по 300-500 изображений на каждый класс, поместите их в zip-архив
# Загрузите архив в Google Colab и не забудьте поменять пути и количество классов (в блоке 3)
!wget https://cdn.freecodecamp.org/project-data/cats-and-dogs/cats_and_dogs.zip
!7z x cats_and_dogs.zip

In [ ]:
# Блок 2. Импорт необходимых библиотек
# Если планируете запуск на домашнем компьютере, не забудьте к Python 3.8 и новее
# поставить библиотеки командой pip install opencv-python, keras, tensorflow, scikit-learn
import cv2, sklearn, os, keras
import numpy as np
import tensorflow as tf
from keras.optimizers import *
from keras.models import Model, Sequential
from keras.layers import *
from keras.utils import *

In [ ]:
# Блок 3. Объявление констант

# Размер входного изображения
size = 256

# Папка с обучающей выборкой
train_dir = "/content/cats_and_dogs/train"

# Цвет изображения
CLR = cv2.IMREAD_COLOR

# Количество классов
classes = 2

In [ ]:
# Блок 4. Функция загрузки изображений
# Все изображения из каталога загружаются в RAM в виде массивов X и Y
# Массив X представляет собой изображения [img_count, size, size, 3]
# Массив Y - индексы классов [img_count]
def create_train_data(train_dir, size, color_mode):
  # Массивы X и Y
  X = []
  Y = []

  #Индекс класса
  pp = 0

  # По всем классам(папкам), и по всем файлам в них
  for p1 in sorted(os.listdir(train_dir)):
    if not p1.startswith('.'):
      for img in os.listdir(os.path.join(train_dir, p1)):
            # Формируем путь к файлу
            path = os.path.join(os.path.join(train_dir, p1), img)
            # Изменяем размер изображения и цветовое пространство
            img = cv2.resize(cv2.imread(path, color_mode), (size, size))
            # Добавляем картинку в обучающую выборку
            X.append(list(np.array(img)))
            # А также её индекс
            Y.append([pp])
      print(pp,p1)
      pp=pp+1

  # Формируем и выгружаем массивы
  Y = np.array(Y)
  X = np.array(X)
  X, Y = sklearn.utils.shuffle(X, Y)
  return (X,Y)

In [ ]:
# Блок 5. Загрузка данных и преобразование выходов
# Для классификационной сети требуется преобразование из формата
# [3] => [0, 0, 0, 1, 0], [0] => [1, 0, 0, 0, 0]
# Это преобразование one-hot Encoding
# ПОдробнее тут: https://habr.com/ru/companies/karuna/articles/769366/
(X, Y) = create_train_data(train_dir, size, CLR)
Y_en = tf.keras.utils.to_categorical(Y, num_classes=classes, dtype='float32')

In [ ]:
# Блок 6. Построение модели сети.
# Это моя базовая модель классификации. Она быстро обучается и работает,
# при этом демонстрируя высокую точность, работает на слоях 6 типов:
# Conv2D, Activation, MaxPooling, Dense, Flatten, Dropout
# О том, как все это устроено можно найти тут: https://keras.io/api/layers/
def build_model(size, classes):
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding="same", input_shape=(size, size, 3)))
    model.add(Conv2D(32, (3, 3), padding="same"))
    model.add(Activation("elu"))
    model.add(MaxPooling2D(pool_size=(4, 4)))
    model.add(Dropout(0.15))

    model.add(Conv2D(32, (3, 3), padding="same"))
    model.add(Activation("elu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.15))

    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("elu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.15))

    model.add(Conv2D(128, (3, 3), padding="same"))
    model.add(Activation("elu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.15))

    model.add(Conv2D(256, (3, 3), padding="same"))
    model.add(Activation("elu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.15))

    model.add(Conv2D(256, (3, 3), padding="same"))
    model.add(Activation("elu"))
    model.add(MaxPooling2D(pool_size=(4, 4)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation("elu"))

    model.add(Dense(classes))
    model.add(Activation("softmax"))

    return model

In [ ]:
# Блок 7. Строим модель и запускаем обучение
# Метод Summary выводит архитектуру модели
# Метод Compile собирает граф сети, добавляя в него необходимые служебные слои для работы оптимизатора
# Для оценки качества работы сеть будет полагаться на функцию bce. Для задач классификации это наилучший вариант
# Метод Fit запускает обучение. Сеть 200 раз прогонит через себя входные данные (X, Y_en) пакетами по 32.
# 20% От выборки не будет участвовать в обучении. Это валидация. На ней будем проверять качество работы
model = build_model(size, classes)
model.summary()
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy', 'mse', 'mae'])
history = model.fit(X,Y_en,epochs=200, batch_size=32, validation_split=0.2)

In [ ]:
# Блок 8. Получение метрик сети в виде графика
# Выводится график для функции потерь (loss = 'bce')
# Тут чем меньше - тем лучше.
# Но если функция начинает расти для графика val_loss, значит пошло переобучение.
# Определите на какой эпохе это началось и в следующий раз запускайте обучение до этой эпохи
from matplotlib import pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Функция потерь')
plt.ylabel('Потери (меньше - лучше)')
plt.xlabel('Число эпох')
plt.legend(['Обучающая', 'Тестовая'], loc='upper left')
plt.yscale('log')
plt.show()

In [ ]:
# Блок 9. Тестирование сети

# Загружаем файл на облако
from google.colab import files
uploaded = files.upload()

# Больше не печатать в экспоненциальном виде
np.set_printoptions(suppress=True)

# Тестирование
for fn in uploaded.keys():
  test = cv2.resize(cv2.imread(fn, CLR), (size, size))
  test = tf.expand_dims(test, axis=0)
  print(model.predict(test))

In [ ]:
# Блок 10. Что дальше?
# Можно попробовать обучить более сложные модели сетей (тут подробное описание всего встроенного зоопарка: https://keras.io/api/applications/)
model = keras.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=(size,size,3),
    pooling=None
)
out = Flatten()(model.output)
out = Dense(units=classes, activation='softmax')(out)
model = Model(inputs=model.input, outputs=out, name='InceptionV3')
#model.summary()
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy', 'mse', 'mae'])
history = model.fit(X,Y_en,epochs=200, batch_size=32, validation_split=0.2)

In [ ]:
# Блок 11. Экспорт модели сети
# А можно попробовать сохранить сеть в формате TFLite. В таком формате она может быть запущена
# на различных мобильных устройствах.
import tensorflow as tf
from keras.models import model_from_json
keras.backend.clear_session()

# Сохраняем веса модели в TensorFlow формате
model.save('model_full.h5')
model = tf.keras.models.load_model('model_full.h5', compile=False)

# Конвертируем модель в TFLIte
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Сохраняем граф модели в файл
with tf.io.gfile.GFile('model.tflite', 'wb') as f:
  f.write(tflite_model)